# セルオートマトン

セルオートマトン（cellular automaton）とは、格子状に広がるセルを用いた離散的計算モデルである。各セルは有限種類の内部状態を持ち、時刻 $t$（離散的なので世代とも呼ばれる）でのセルの状態は、一定の規則によって、前時刻での自分自身や周りのセルの状態から決定される[<sup id="cite_ref-1">[1]</sup>](#cite_note-1)。非常に単純化されたモデルではあるが、複雑で非自明な結果を与え得ることが知られている。

1980年代、スティーブン・ウルフラムは、今回取り上げる基本セルオートマトン（elementary cellular automaton）の研究から、セルオートマトンをその振る舞いによって四つのクラスに分類している[<sup id="cite_ref-2">[2]</sup>](#cite_note-2)：
- クラス 1：すべてのセルが一様な状態へと収束する（すべて生きている、すべて死ぬなど）。
- クラス 2：単純な挙動の一つへと落ち着く（ずっと生き続ける、ずっと死んでいる、周期的に移り変わるなど）。
- クラス 3：ランダムな振る舞いを見せる。一時的に局所的な構造が作られるかもしれないが、すぐに露と消える。
- クラス 4：秩序状態とランダムな状態の混合である。局所的な構造が移動しながら相互作用を起こす。

## 基本セルオートマトン

もっとも簡単なセルオートマトンとして、一次元にセルが並び、各セルは二つの状態（0 か 1）[<sup id="cite_ref-3">[3]</sup>](#cite_note-3)をとり、自分自身と両側に接している隣の（合わせて三つの）セルの状態によって、次の時刻の状態が決まるというものを考える。このようなセルオートマトンは、基本セルオートマトン（elementary cellular automaton）と呼ばれる。

最近傍三つのセルの状態は $2^3 = 8$ パターンあるので、各パターンによって中心セルがどのように変化するかを決めることで規則が定義できる。例えば、図 1 のように中心セルの状態が変化する規則を考えよう。

![ルール 250](images/rule250.png)
図 1. ルール 250 によるセルの遷移規則

最近傍三つのセル 8 パターンの順番をこのように固定したとき（最近傍三つのセルを 2 進数として読むと、右から順に $000_2 = 0$、$001_2 = 1$、$010_2 = 2$、$011_2 = 3$、$\dots$、一番左が $111_2 = 7$ である）、この規則をウルフラムの命名法にしたがってルール 250 と呼ぶ。なぜなら、$11111010_2 = 250$ だからである。このような規則は、ルール 0（$00000000_2 = 0$）からルール 255（$11111111_2 = 255$）までの $2^8 = 256$ 種類がある。

このような基本セルオートマトンを実行するプログラムを考えよう。計算機上で無限に長い一次元セルを表現することは困難であるので、セルの個数は有限とし、両端のセルはつながっているものとする（周期境界条件）。すると、一次元に並んだセルは、$0$ か $1$ の要素が入ったリストによって表せるだろう。セルの初期状態について、とりあえずは、確率 $1/2$ でランダムに $0$ か $1$ を設定しよう。次の世代のセルの状態は、上述した規則によって、前の世代のセルの状態から決定する。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random


def random_initialize(size):
    # 長さsizeのランダムに0か1で初期化されたリストを返す関数
    cells = [0] * size
    for i in range(size):
        if random.random() < 0.5:
            cells[i] = 1
    return cells


def cellular_automaton(rule, initialize=random_initialize, size=100, tmax=200, figscale=0.08):
    # セルオートマトンを実行して描画する関数
    #
    # rule: ルールを表すリスト（長さ8、各要素は0か1）
    # initialize: 初期化されたセルをリストで与える関数
    # size: セルのサイズ
    # tmax: 最終時刻
    # figscale: 描画する図のサイズを調節するスケーリング因子
    
    # 各世代のセルの状態を保存するリスト
    generations = []
    
    # 初期状態
    cells = initialize(size)
    generations.append(cells)
    
    for _ in range(tmax):
        next_cells = [0] * size  # 次の世代のセル
        
        for i in range(size):
            # 周期境界条件（leftとrightは必ず0からsize-1の間）
            left = i - 1
            right = i + 1
            if left < 0:
                left += size
            if right >= size:
                right -= size
            
            # 最近傍三つのセルの状態を2進数とみなして読み取る(kは0から7)
            k = cells[left] * 4 + cells[i] * 2 + cells[right]
            # 対応する状態をルールから読み取る
            next_cells[i] = rule[k]
        
        # セルを更新
        cells = next_cells
        generations.append(cells)
    
    # 描画する
    fig, ax = plt.subplots(figsize=(size * figscale, tmax * figscale))  # 図のサイズを調節している
    ax.matshow(generations, vmin=0, vmax=1)
    plt.show()


# ルール250のセルオートマトン
cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1])

上の例題プログラムの `cellular_automaton` 関数に渡す `rule` という引数は、`rule[0]` がセルの状態 $000_2$ に対する中央セルの次世代での値、`rule[1]` が $001_2$ の値、`rule[2]` が $010_2$ の値、$\dots$、`rule[7]` が $111_2$ の値となるようなリストである。よって、ルール 250 に対しては `[0, 1, 0, 1, 1, 1, 1, 1]` となる（リストとして `[...]` のように書くと、図 1 と左右逆になることに注意）。

`cellular_automaton` 関数では、各世代のセルの状態（`cells`）をすべて `generations` というリストに保存している。各世代のセルの状態はそれ自体が（一次元）リストであるので、`generations` は二次元のリストとなっている。これを [Axes オブジェクト](https://matplotlib.org/api/axes_api.html#the-axes-class)の [matshow メソッド](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.matshow.html) に渡すことで、二次元リストに格納された内容を描画している。縦軸が時間軸である。暗い部分は $0$、明るい部分は $1$ に対応する。

実行すると分かるように、ルール 250 はすぐ一様な状態（すべてのセルが $1$）に収束し、クラス 1 に分類される。

## 練習問題

(1) 次に挙げるルールに基づく基本セルオートマトンを実行し、クラス 2、3、4 の特徴も確認しよう。
- ルール 4：クラス 2 の「周期的」なパターンの例。
- ルール 30：クラス 3 の「ランダム」なパターンの例[<sup id="cite_ref-4">[4]</sup>](#cite_note-4)。
- ルール 110：クラス 4 の「複雑な局所構造」を持つパターンの例[<sup id="cite_ref-5">[5]</sup>](#cite_note-5)[<sup id="cite_ref-6">[6]</sup>](#cite_note-6)。

## 演習課題

上の例題で定義された `random_initialize` と `cellular_automaton` について、**そのまま変更せずに使うのなら**解答セルにそのコピーを書かなくてもよいこととする。ただし、一部でも変更するのであれば解答セル内に定義し直すこと。解答セル内に定義がなければ、上の例題での定義であるとみなす。

(1) 例題のようにランダムな初期配置で、次のルールの基本セルオートマトンを実行せよ。また、それらをクラス 1 から 4 に分類せよ。

(a) ルール 45

(b) ルール 54

(c) ルール 78

(d) ルール 136

In [ ]:
# 課題解答12.1  <-- 提出する際に、この行を必ず含めること。

# ここで(a)から(d)のルールでセルオートマトンを実行してください。
# （ルール250になっているので、適切に書き換えること。）

cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1])  # ルール45に書き換えること
cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1])  # ルール54に書き換えること
cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1])  # ルール78に書き換えること
cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1])  # ルール136に書き換えること

# ここに（Pythonのコメントとして）分類を答えてください。
#
# (a) ルール45  -> クラス?
# (b) ルール54  -> クラス?
# (c) ルール78  -> クラス?
# (d) ルール136 -> クラス?

(2) 初期配置として、（ほぼ）中心のセル `cells[size // 2]` のみに $1$ を置き、あとはすべて $0$ となるようにして、ルール 90 のセルオートマトンを実行せよ。

In [ ]:
# 課題解答12.2  <-- 提出する際に、この行を必ず含めること。




## 発展課題

余裕があればやってみてください。

(3) $0$ から $255$ までの整数 $n$ を入力として、今回使った `cellular_automaton` 関数の `rule` 引数に渡せるようなリストを返す関数 `make_rule` を書け。

In [ ]:
# 課題解答12.3  <-- 提出する際に、この行を必ず含めること。


def make_rule(n):  # ルールnの遷移規則を表すリストを返す関数
    ...  # いい感じに書き換えてください。


# ルール250のセルオートマトン
cellular_automaton(rule=make_rule(250))

## 脚注

<span id="cite_note-1">1.</span> [^](#cite_ref-1)
一般には、格子は何次元でもよく、また正方格子である必要もない。例えば、格子の広がる空間がトーラス（ドーナツ状）でもよい。さらに「周りのセル」の定義もさまざまである。

<span id="cite_note-2">2.</span> [^](#cite_ref-2)
Stephen Wolfram, *A New Kind of Science*, Wolfram Media Inc. (2002) ISBN 978-1-57955-008-0, [[open access]](https://www.wolframscience.com/nks/).

<span id="cite_note-3">3.</span> [^](#cite_ref-3)
0 か 1 のセルの状態をそれぞれ「死んでいる」「生きている」と表現することがある。

<span id="cite_note-4">4.</span> [^](#cite_ref-4)
ある種の貝の貝殻にルール 30 に似た模様が現れることが知られている。

<span id="cite_note-5">5.</span> [^](#cite_ref-5)
局所構造やその相互作用については、`size` や `tmax` を大きくしてみると分かりやすいだろう。

<span id="cite_note-6">6.</span> [^](#cite_ref-6)
ルール 110 はチューリング完全であることが証明されている。つまり、原理的には、コンピューターで計算できること、シミュレーションできることがすべてできる。